# Horovod Distributed Training with SageMaker TensorFlow script mode.

Horovod is a distributed training framework based on MPI. You can find more details at [Horovod README](https://github.com/uber/horovod).

Horovod Distributed Training can be perfomed on SageMaker using the SageMaker Tensorflow container. If MPI is enabled at the training job creation time, then SageMaker creates the MPI environment and executes the `mpirun` command to execute the training script. More details on how to configure mpi settings in training job can be found below.

In this example notebook, we will create a MNIST horovod training job.

## Set up the environment

We will fetch the `IAM` role this notebook is running in and pass the same role to the TensorFlow estimator which SageMaker will assume to fetch data and perform training.

In [ ]:
import sagemaker
import os
from sagemaker.utils import sagemaker_timestamp
from sagemaker.tensorflow import TensorFlow
from sagemaker import get_execution_role

sagemaker_session = sagemaker.Session()

default_s3_bucket = sagemaker_session.default_bucket()
sagemaker_iam_role = get_execution_role()

train_script = "mnist_hvd.py"
instance_count = 2

## Prepare Data for training

In this, we will download the mnist dataset to the local `/tmp/data/` directory which is then uploaded to the s3 followed by cleaning up `/tmp/data/`. 

In [ ]:
import os
import shutil

import numpy as np

import keras
from keras.datasets import mnist
(x_train, y_train), (x_test, y_test) = mnist.load_data()

s3_train_path = "s3://{}/mnist/train.npz".format(default_s3_bucket)
s3_test_path = "s3://{}/mnist/test.npz".format(default_s3_bucket)

# Create local directory
! mkdir -p /tmp/data/mnist_train
! mkdir -p /tmp/data/mnist_test

# Save data locally
np.savez('/tmp/data/mnist_train/train.npz', data=x_train, labels=y_train)
np.savez('/tmp/data/mnist_test/test.npz', data=x_test, labels=y_test)

# Upload the dataset to s3
! aws s3 cp /tmp/data/mnist_train/train.npz $s3_train_path
! aws s3 cp /tmp/data/mnist_test/test.npz $s3_test_path

print('training data at ', s3_train_path)
print('test data at ', s3_test_path)
! rm -rf /tmp/data

## Construct a script for horovod distributed training

In this example we are using the [Keras MNIST horovod example](https://github.com/uber/horovod/blob/master/examples/keras_mnist.py) from the horovod github repository.

In order to run this script we have to make following modifications:

### 1. Accept `--model_dir` command-line argument
Modify script to accept `--model_dir` as command-line argument which will define the directory path (i.e. `/opt/ml/model/`) where the output model should be saved. As Sagemaker destroys the complete cluster at the end of training, saving the model to `/opt/ml/model/` directory preserves the trained model from getting lost as SageMaker at the end of trainig pushes all the data in `/opt/ml/model/` to s3. 

This also allows the SageMaker training to integrate with other SageMaker services such as Inference and also allows you to host the trained model outside SageMaker.

Here is the code that needs to be added to script:

```
parser = argparse.ArgumentParser()
parser.add_argument('--model_dir', type=str)
```

More details can be found [here](https://github.com/aws/sagemaker-containers/blob/master/README.md).

### 2. Load train and test data

Local directory path where the train and test are downloaded can be fetched by reading the environment variabled `SM_CHANNEL_TRAIN` and `SM_CHANNEL_TEST` respectively.
After fetching the directory path we can load the data in memory.

Here is the code:

```
x_train = np.load(os.path.join(os.environ['SM_CHANNEL_TRAIN'], 'train.npz'))['data']
y_train = np.load(os.path.join(os.environ['SM_CHANNEL_TRAIN'], 'train.npz'))['labels']

x_test = np.load(os.path.join(os.environ['SM_CHANNEL_TEST'], 'test.npz'))['data']
y_test = np.load(os.path.join(os.environ['SM_CHANNEL_TEST'], 'test.npz'))['labels']
```

List of all environemnt variables set by SageMaker which are accessible inside training script can be found [here](https://github.com/aws/sagemaker-containers/blob/master/README.md).

### 3. Save model only at master node

As in horovod the training is distributed to multiple nodes the model should only be saved by the master for which we have to add following lines of code:

```
# Horovod: Save model only on worker 0 (i.e. master)
if hvd.rank() == 0:
    saved_model_path = tf.contrib.saved_model.save_keras_model(model, args.model_dir)
```

### Training script

Here is the final training script.

In [ ]:
!cat 'mnist_hvd.py'

## Test locally using SageMaker Python SDK TensorFlow Estimator

You can use the SageMaker Python SDK TensorFlow estimator to easily train locally and in SageMaker.

This notebook shows how to use the SageMaker Python SDK to run your code in a local container before deploying to SageMaker's managed training or hosting environments. Just change your estimator's `train_instance_type` to `local` or `local_gpu`. For more information, see: https://github.com/aws/sagemaker-python-sdk#local-mode.

In order to use this feature you'll need to install docker-compose (and nvidia-docker if training with a GPU). Running following script will install docker-compose or nvidia-docker-compose and configure the notebook environment for you.

**Note**: You can only run a single local notebook at a time.

In [ ]:
!/bin/bash ./setup.sh

To train locally, you set train_instance_type to local:

In [ ]:
train_instance_type='local'

MPI environment for Horovod can be configured by following flags in `mpi` field of `distribution` dict passed to TensorFlow estimator :

* ``enabled (bool)``: If set to ``True``, the MPI setup is performed and ``mpirun`` command is executed.
* ``processes_per_host (int) [Optional]``: Number of processes MPI should launch on each host. Note, this should not be greater than the available slots on the selected instance type. This flag should be set for the multi-cpu/gpu training.
* ``custom_mpi_options (str) [Optional]``: Any mpirun flag(s) can be passed in this field that will be added to the mpirun command executed by SageMaker to launch distributed horovod training.

You can find more details on `distribution` dictionary in SageMaker python SDK [README](https://github.com/aws/sagemaker-python-sdk/blob/v1.17.3/src/sagemaker/tensorflow/README.rst).

Let's start with just enabling MPI

In [ ]:
distributions = {'mpi': {'enabled': True}}

Now, we create the Tensorflow estimator passing the `train_instance_type` and `distribution`

In [ ]:
estimator_local = TensorFlow(entry_point=train_script,
                       role=sagemaker_iam_role,
                       train_instance_count=instance_count,
                       train_instance_type=train_instance_type,
                       script_mode=True,
                       framework_version='1.12',
                       distributions=distributions,
                       base_job_name='hvd-mnist-local')

Call `fit()` to start the local training 

In [ ]:
estimator_local.fit({"train":s3_train_path, "test":s3_test_path})

## Train in SageMaker

After you test the training job locally, now run it on SageMaker:

First, change the instance type from `local` to the valid ec2 instance type, say `ml.c4.xlarge` 

In [ ]:
train_instance_type='ml.c4.xlarge'

You can also provide your custom MPI options by passing in the `custom_mpi_options` field of `distribution` dictionary that will be added to the `mpirun` command executed by SageMaker:

In [ ]:
distributions = {'mpi': {'enabled': True, "custom_mpi_options": "-verbose --NCCL_DEBUG=INFO"}}

Now, we create the Tensorflow estimator passing the `train_instance_type` and `distribution` to launche training in sagemaker.

In [ ]:
estimator = TensorFlow(entry_point=train_script,
                       role=sagemaker_iam_role,
                       train_instance_count=instance_count,
                       train_instance_type=train_instance_type,
                       script_mode=True,
                       framework_version='1.12',
                       distributions=distributions,
                       base_job_name='hvd-mnist')

Call `fit()` to start the training

In [ ]:
estimator.fit({"train":s3_train_path, "test":s3_test_path})

##  Horovod training in SageMaker using multiple CPU/GPU

To enable mulitiple CPU/GPU horovod training, you have to set the `processes_per_host` field in `mpi` section of `distribution` dictionary to the desired value of processes that will be executed per instance.

In [ ]:
distributions = {'mpi': {'enabled': True, "processes_per_host": 2}}

Now, we create the Tensorflow estimator passing the `train_instance_type` and `distribution`

In [ ]:
estimator = TensorFlow(entry_point=train_script,
                       role=sagemaker_iam_role,
                       train_instance_count=instance_count,
                       train_instance_type=train_instance_type,
                       script_mode=True,
                       framework_version='1.12',
                       distributions=distributions,
                       base_job_name='hvd-mnist-multi-cpu')

Call `fit()` to start the training

In [ ]:
estimator.fit({"train":s3_train_path, "test":s3_test_path})

## Improving horovod training performance on SageMaker

By performing horovod training inside VPC improves the network latency between nodes leading to high performance and stability of Horovod training jobs.

Detailed explanation on how to configure VPC for SageMaker training can be found [here](https://github.com/aws/sagemaker-python-sdk#secure-training-and-inference-with-vpc).

### Setup VPC infrastructure
We will setup following resources as part of VPC stack:
* `VPC`: AWS Virtual private cloud with CIDR block.
* `Subnets`: Two subnets with the CIDR blocks `10.0.0.0/24` and `10.0.1.0/24`
* `Security Group`: Defining the open ingress and egress ports, such as TCP.
* `VpcEndpoint`: S3 Vpc endpoint allowing sagemaker's vpc cluster to dosenload data from S3.
* `Route Table`: Defining routes and is tied to subnets and VPC.

Complete cloud formation template for setting up the VPC stack can be seen [here](./vpc_infra_cfn.json).

In [ ]:
import boto3
from botocore.exceptions import ClientError
from time import sleep

def create_vpn_infra(stack_name="hvdvpcstack"):
    cfn = boto3.client("cloudformation")

    cfn_template = open("vpc_infra_cfn.json", "r").read()
    
    try:
        vpn_stack = cfn.create_stack(StackName=(stack_name),
                                     TemplateBody=cfn_template)
    except ClientError as e:
        if e.response['Error']['Code'] == 'AlreadyExistsException':
            print("Stack: {} already exists, so skipping stack creation.".format(stack_name))
        else:
            print("Unexpected error: %s" % e)
            raise e

    describe_stack = cfn.describe_stacks(StackName=stack_name)["Stacks"][0]

    while describe_stack["StackStatus"] == "CREATE_IN_PROGRESS":
        describe_stack = cfn.describe_stacks(StackName=stack_name)["Stacks"][0]
        sleep(0.5)

    if describe_stack["StackStatus"] != "CREATE_COMPLETE":
        raise ValueError("Stack creation failed in state: {}".format(describe_stack["StackStatus"]))

    print("Stack: {} created successfully with status: {}".format(stack_name, describe_stack["StackStatus"]))

    subnets = []
    security_groups = []

    for output_field in describe_stack["Outputs"]:

        if output_field["OutputKey"] == "SecurityGroupId":
            security_groups.append(output_field["OutputValue"])
        if output_field["OutputKey"] == "Subnet1Id" or output_field["OutputKey"] == "Subnet2Id":
            subnets.append(output_field["OutputValue"])

    return subnets, security_groups


subnets, security_groups = create_vpn_infra()
print("Subnets: {}".format(subnets))
print("Security Groups: {}".format(security_groups))

### VPC training in SageMaker
Now, we create the Tensorflow estimator passing the train_instance_type and distribution

In [ ]:
estimator = TensorFlow(entry_point=train_script,
                       role=sagemaker_iam_role,
                       train_instance_count=instance_count,
                       train_instance_type=train_instance_type,
                       script_mode=True,
                       framework_version='1.12',
                       distributions=distributions,
                       security_group_ids=['sg-0919a36a89a15222f'],
                       subnets=['subnet-0c07198f3eb022ede', 'subnet-055b2819caae2fd1f'],
                       base_job_name='hvd-mnist-vpc')

Call `fit()` to start the training

In [ ]:
estimator.fit({"train":s3_train_path, "test":s3_test_path})

Once training is completed, the saved model can be hosted using the TensorFlow Serving on SageMaker. Example notebook for same can be found [here](https://github.com/awslabs/amazon-sagemaker-examples/blob/master/sagemaker-python-sdk/tensorflow_serving_container/tensorflow_serving_container.ipynb).

## Reference Links:
* [SageMaker Container MPI Support.](https://github.com/aws/sagemaker-containers/blob/master/src/sagemaker_containers/_mpi.py)
* [Horovod Official Documentation](https://github.com/uber/horovod)
* [SageMaker Tensorflow script mode example.](https://github.com/awslabs/amazon-sagemaker-examples/blob/master/sagemaker-python-sdk/tensorflow_script_mode_quickstart/tensorflow_script_mode_quickstart.ipynb)